In [ ]:
#PASO 1 — Leer los .json y preparar los tweet
import json

def load_hateval_json(file_path):
    texts = []
    labels = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data = json.loads(line.strip())
                texts.append(data["text"])
                labels.append(int(data["klass"]))
            except Exception as e:
                print(f"Error leyendo línea: {e}")
    
    return texts, labels


In [ ]:
#Paso 2 — Cargar los datos de entrenamiento y prueba
X_train_es, y_train_es = load_hateval_json("hateval_es_train_spanish.json")
X_test_es, y_test_es = load_hateval_json("hateval_es_test_spanish.json")

X_train_in, y_train_in = load_hateval_json("hateval_en_train_ingles.json")
X_test_in, y_test_in = load_hateval_json("hateval_en_test_ingles.json")


In [ ]:
#Comprobar las cargas
print("ES train:", len(X_train_es))
print("ES test:", len(X_test_es))
print("EN train:", len(X_train_in))
print("EN test:", len(X_test_in))



ES train: 4500
ES test: 500
EN train: 9000
EN test: 1000


In [ ]:
#librerías para preprocesamiento de texto  y tokenización
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer


In [ ]:
# Descargar recursos necesarios UNA SOLA VEZ
nltk.download('stopwords')

In [13]:
#PASO 3 — Preprocesamiento de texto y tokenización 

# Configurar herramientas por idioma
stopwords_es = set(stopwords.words('spanish'))
stopwords_en = set(stopwords.words('english'))
stemmer_es = SnowballStemmer('spanish')
stemmer_en = SnowballStemmer('english')

# Funciones de preprocesamiento
def limpiar_texto(texto):
    """
    Limpia el texto antes de tokenizar:
    - Convierte a minúsculas
    - Elimina URLs, menciones, hashtags, emojis y caracteres raros
    """
    texto = texto.lower()
    texto = re.sub(r"http\S+|www\S+|https\S+", "", texto)  # URLs
    texto = re.sub(r"@\w+", "", texto)  # menciones
    texto = re.sub(r"#", "", texto)  # hashtags
    texto = re.sub(r"[^a-záéíóúñü\s]", "", texto)  # solo letras
    return texto

# Función para preprocesar texto en español
def preprocesar_espaniol(texto):
    texto = limpiar_texto(texto)
    tokens = texto.split()
    tokens = [t for t in tokens if t not in stopwords_es]  # eliminar stopwords
    tokens = [stemmer_es.stem(t) for t in tokens]  # aplicar stemming
    return " ".join(tokens)

# Función para preprocesar texto en inglés
def preprocesar_ingles(texto):
    texto = limpiar_texto(texto)
    tokens = texto.split()
    tokens = [t for t in tokens if t not in stopwords_en]  # eliminar stopwords
    tokens = [stemmer_en.stem(t) for t in tokens]  # aplicar stemming
    return " ".join(tokens)


In [ ]:
# Preprocesar los conjuntos de datos
X_entrenamiento_es = [preprocesar_espaniol(t) for t in X_train_es]
X_prueba_es = [preprocesar_espaniol(t) for t in X_test_es]

X_entrenamiento_en = [preprocesar_ingles(t) for t in X_train_in]
X_prueba_en = [preprocesar_ingles(t) for t in X_test_in]


In [15]:
#PASO 4 — Vectorización de texto dependiendo del método elegido
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def vectorizar_textos(textos_entrenamiento, textos_prueba, metodo='tfidf', ngramas=(1,1)):
    """
    Convierte los textos en vectores numéricos.
    
    Parámetros:
    textos_entrenamiento: lista de strings de entrenamiento
    textos_prueba: lista de strings de prueba
    metodo: 'tf' para frecuencia de términos, 'tfidf' para TF-IDF
    ngramas: tupla para definir ngramas, ej. (1,1) unigramas, (1,2) uni+bi
    
    Retorna:
    X_entrenamiento_vect: matriz de entrenamiento
    X_prueba_vect: matriz de prueba
    vectorizador: objeto vectorizador para transformar nuevos datos si es necesario
    """
    if metodo == 'tf':
        vectorizador = CountVectorizer(ngram_range=ngramas)
    elif metodo == 'tfidf':
        vectorizador = TfidfVectorizer(ngram_range=ngramas)
    else:
        raise ValueError("Método debe ser 'tf' o 'tfidf'")
    
    # Ajustar el vectorizador solo con los datos de entrenamiento
    X_entrenamiento_vect = vectorizador.fit_transform(textos_entrenamiento)
    X_prueba_vect = vectorizador.transform(textos_prueba)
    
    return X_entrenamiento_vect.toarray(), X_prueba_vect.toarray(), vectorizador


In [21]:
# Ejemplo de vectorización para español usando TF-IDF y unigramas
X_entrenamiento_es_vect, X_prueba_es_vect, vectorizador_es = vectorizar_textos(
    X_entrenamiento_es, X_prueba_es, metodo='tfidf', ngramas=(1,1)
)


In [22]:
# Ejemplo de vectorización para inglés usando TF-IDF y unigramas
X_entrenamiento_en_vect, X_prueba_en_vect, vectorizador_en = vectorizar_textos(
    X_entrenamiento_en, X_prueba_en, metodo='tfidf', ngramas=(1,1)
)


In [16]:
#CLASE MLP_TODO.PY Y FUNCIONES AUXILIARES
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
from time import time

# ====================================================
# Funciones para activación y  su derivada
# ====================================================

# Función de activación sigmoide
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivada de la sigmoide
def sigmoid_derivative(x):
    # return sigmoid(x) * (1 - sigmoid(x))
    return x * (1 - x)

# ====================================================
# Funciones para manejo de la semilla
# ====================================================

# Establece la semilla para la generación de números aleatorios
def seed(random_state=33):
    np.random.seed(random_state)
    random.seed(random_state)

# ====================================================
# Funciones para inicialización y normalización
# ====================================================

# Inicialización Xavier
def xavier_initialization(input_size, output_size): 
    return np.random.randn(input_size, output_size) * np.sqrt(1 / input_size)
# Inicialización normal
def normal_initialization(input_size, output_size):
    return np.random.randn(input_size, output_size)
# Normalización Z-score
def zscore_normalization(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X_norm = (X - mean) / std
    return X_norm

#Función para crear minibatches
def create_minibatches(X, y, batch_size):
    """
    Genera los lotes de datos (batchs) de acuerdo al parámetro batch_size de forma aleatoria para el procesamiento. 
    """
    n_samples = X.shape[0]
    indices = np.random.permutation(n_samples)  # Mezcla los índices aleatoriamente
    X_shuffled, y_shuffled = X[indices], y[indices]  # Reordena X e y según los índices aleatorios
    
    # Divide los datos en minibatches
    for X_batch, y_batch in zip(np.array_split(X_shuffled, np.ceil(n_samples / batch_size)), 
                                np.array_split(y_shuffled, np.ceil(n_samples / batch_size))):
        yield X_batch, y_batch

    
class MLP_TODO:
    def __init__(self, num_entradas, num_neuronas_ocultas, num_salidas, epochs, batch_size=128, learning_rate=0.2, normalizacion="none",inicializacion="xavier",random_state=42):

        # ====================================================
        # Inicialización general del modelo
        # ====================================================

        # 🔹 NUEVO: Usar el parámetro random_state recibido para controlar la semilla
        seed(33)
        self.random_state = random_state  # 🔹 NUEVO: guardar la semilla para usarla también en create_minibatches

        # Definir la tasa de aprendizaje
        self.learning_rate = learning_rate
        # Definir el número de épocas
        self.epochs = epochs
        # Definir el tamaño del batch de procesamiento
        self.batch_size = batch_size
        # Definir el tipo de normalización
        self.normalizacion = normalizacion
        # Definir el tipo de inicialización
        self.inicializacion = inicializacion
        # definir las 
        self.num_neuronas_ocultas = num_neuronas_ocultas

        # Inicialización de pesos y bias
        self.W1 = self.inicializar_pesos(num_entradas, self.num_neuronas_ocultas) # Pesos entre capa de entrada y capa oculta
        self.b1 = np.zeros((1, self.num_neuronas_ocultas))   # Bias de la capa oculta
        self.W2 = self.inicializar_pesos(self.num_neuronas_ocultas,num_salidas)  # Pesos entre capa oculta y capa de salida
        self.b2 = np.zeros((1, num_salidas)) # Bias de la capa de salida

        # Historial de errores
        self.errores_history = []
        # Historial de accuracy
        self.accuracy_history = []

    # ====================================================
    # Funciones para forward, backward, update, predict y train
    # ====================================================

    def forward(self, X):
        #implementar el forward pass
        #----------------------------------------------
        # 1. Propagación hacia adelante (Forward pass)
        #----------------------------------------------
        # Calcular la suma ponderada Z (z_c1) para la capa oculta 
        self.X = X
        self.z_c1 = X@self.W1 + self.b1
        #Calcular la activación de la capa oculta usando la función sigmoide
        self.a_c1 = sigmoid(self.z_c1)  # Activación capa oculta
        #Calcular la suma ponderada Z (z_c2)  para la capa de salida 
        self.z_c2  = self.a_c1 @ self.W2 + self.b2
        #Calcular la activación de la capa de salida usando la función sigmoide
        y_pred = sigmoid(self.z_c2)  # Activación capa salida
        return y_pred
    

    def loss_function_MSE(self, y_pred, y):
        #----------------------------------------------
        # 2. Cálculo del error con MSE
        #----------------------------------------------
        #Calcular el error cuadrático medio (MSE)
        self.y_pred = y_pred
        self.y = y
        error = 0.5 * np.mean((y_pred - y) ** 2)
        return error
    

    def backward(self):
        #implementar el backward pass
        # calcular los gradientes para la arquitectura de la figura anterior
        #----------------------------------------------
        # 3. Propagación hacia atrás (Backward pass)
        #----------------------------------------------
        
        #----------------------------------------------
        # Gradiente de la salida
        #----------------------------------------------
        #Calcular la derivada del error con respecto a la salida y
        dE_dy_pred = (self.y_pred - self.y)  # Derivada del error respecto a la predicción con  N ejemplos
        #Calcular la derivada de la activación de la salida con respecto a z_c2 
        d_y_pred_d_zc2 = sigmoid_derivative(self.y_pred)
        #Calcular delta de la capa de salida
        delta_c2 = dE_dy_pred * d_y_pred_d_zc2  # (N, 1)

        #----------------------------------------------
        # Gradiente en la capa oculta
        #----------------------------------------------
        # calcular la derivada de las suma ponderada respecto a las activaciones de la capa 1
        d_zc2_d_a_c1 = self.W2  
        #Propagar el error hacia la capa oculta, calcular deltas de la capa 1
        delta_c1 = delta_c2 @ d_zc2_d_a_c1.T * sigmoid_derivative(self.a_c1)  

        #calcula el gradiente de la función de error respecto a los pesos de la capa 2
        self.dE_dW2 = self.a_c1.T @ delta_c2
        self.dE_db2 = np.sum(delta_c2, axis=0, keepdims=True)
        self.dE_dW1 = self.X.T @ delta_c1 
        self.dE_db1 = np.sum(delta_c1, axis=0, keepdims=True) 


    def update(self):  # Ejecución de la actualización de paramámetros
        #implementar la actualización de los pesos y el bias
        #----------------------------------------------
        # Actualización de pesos de la capa de salida
        #---------------------------------------------- 
        #Actualizar los pesos y bias de la capa de salida
        self.W2 = self.W2 - self.dE_dW2 * self.learning_rate
        self.b2 = self.b2 - self.dE_db2 * self.learning_rate
        #----------------------------------------------
        # Actuailzación de pesos de la capa oculta
        #----------------------------------------------
        #calcula el gradiente de la función de error respecto a los pesos de la capa 1
        self.W1 = self.W1 - self.dE_dW1 * self.learning_rate
        self.b1 = self.b1 - self.dE_db1 * self.learning_rate

    def predict(self, X):  # Predecir la categoría para datos nuevos
        # TODO: implementar la predicción 
        y_pred = self.forward(X)
        # Obtener la clase para el clasificador binario
        y_pred = np.where(y_pred >= 0.5, 1, 0)
        return y_pred

    def train(self, X, Y):
        #implementar el entrenamiento de la red
            # 🔹 Normalizar los datos según el tipo configurado
        X = self.normalize(X)
        for epoch in range(self.epochs):
            num_batch = 0
            epoch_error  = 0
            for X_batch, y_batch in create_minibatches(X, Y, self.batch_size):
                y_pred = self.forward(X_batch)
                error = self.loss_function_MSE(y_pred, y_batch)
                epoch_error += error    
                self.backward() # cálculo de los gradientes
                self.update() # actualización de los pesos y bias
                num_batch += 1
                # Imprimir el error cada N épocas
                if epoch % 100 == 0:
                    print(f"Época {epoch}, Error batch {num_batch}: {error}")
            # Guardar el error promedio de la época
            self.errores_history.append(epoch_error/num_batch)
            #Calcular Accuracy en todo el dataset
            acc_epoch = self.accuracy(X, Y)
            self.accuracy_history.append(acc_epoch)
            # Imprimir el error y accuracy cada N épocas
            if epoch % 100 == 0:
                    print(f"Época {epoch}, Error: {epoch_error/num_batch}%")

    
    # ====================================================
    # Funciones para inicialización, normalización y accuracy
    # ====================================================
    # Normalización de los datos
    def normalize(self, X):
        if self.normalizacion == "z-score":
            return zscore_normalization(X)  # 🔹 Llamada a la función existente
        else:  # sin normalizar
            return X
        
    # Inicialización de los pesos  
    def inicializar_pesos(self, tamaño_entrada, tamaño_salida):
        if self.inicializacion == "xavier":
            return xavier_initialization(tamaño_entrada, tamaño_salida)
        elif self.inicializacion == "normal":
            return normal_initialization(tamaño_entrada, tamaño_salida)
        else:
            raise ValueError("Tipo de inicialización no soportado")
    # Cálculo de accuracy    
    def accuracy(self, X, y):
        y_pred = self.predict(X)
        acc = np.mean(y_pred == y)  # compara predicciones con valores reales
        return acc
    
    

In [18]:
#Necesitamos reshappear los vectores etiquetas
#Convertir tus labels (y_train, y_test) a formato compatible
#MLP espera arrays de N×1, no listas de enteros.
y_train_es = np.array(y_train_es).reshape(-1,1)
y_test_es = np.array(y_test_es).reshape(-1,1)
y_train_en = np.array(y_train_in).reshape(-1,1)
y_test_en = np.array(y_test_in).reshape(-1,1)


In [23]:
#Ejemplo de creación y entrenamiento del MLP para español
mlp_es = MLP_TODO(
    num_entradas=X_entrenamiento_es_vect.shape[1],
    num_neuronas_ocultas=64,
    num_salidas=1,
    epochs=300,
    batch_size=32,
    learning_rate=0.1,
    normalizacion="none",
    inicializacion="xavier"
)

mlp_es.train(X_entrenamiento_es_vect, y_train_es)


Época 0, Error batch 1: 0.12258864110527798
Época 0, Error batch 2: 0.12547158331657152
Época 0, Error batch 3: 0.11374285018796809
Época 0, Error batch 4: 0.13111889198647653
Época 0, Error batch 5: 0.17712875986431312
Época 0, Error batch 6: 0.17455272258982013
Época 0, Error batch 7: 0.12131263681900087
Época 0, Error batch 8: 0.11302445490276229
Época 0, Error batch 9: 0.12489577525597682
Época 0, Error batch 10: 0.1253095132423369
Época 0, Error batch 11: 0.11029598387435884
Época 0, Error batch 12: 0.1450444744084053
Época 0, Error batch 13: 0.2199631709167975
Época 0, Error batch 14: 0.15494323008077254
Época 0, Error batch 15: 0.16942218183773294
Época 0, Error batch 16: 0.17702404541087646
Época 0, Error batch 17: 0.1117343669127069
Época 0, Error batch 18: 0.1285659221714199
Época 0, Error batch 19: 0.12521872406926182
Época 0, Error batch 20: 0.13746868446204585
Época 0, Error batch 21: 0.1996431847243984
Época 0, Error batch 22: 0.12811375076531514
Época 0, Error batch 23: 

In [24]:
#Evaluar el modelo en el conjunto de prueba
predicciones_es = mlp_es.predict(X_prueba_es_vect)
acc_es = mlp_es.accuracy(X_prueba_es_vect, y_test_es)
print("Accuracy español:", acc_es)


Accuracy español: 0.712


In [ ]:
# ====================================================
# Ejemplo de Entrenamiento y evaluación del MLP para Hate Speech - Español e Inglés
# ====================================================

import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# --- Convertir labels a formato N x 1 ---
y_train_es = np.array(y_train_es).reshape(-1,1)
y_test_es = np.array(y_test_es).reshape(-1,1)
y_train_en = np.array(y_train_en).reshape(-1,1)
y_test_en = np.array(y_test_en).reshape(-1,1)

# ====================================================
# FUNCIONES AUXILIARES PARA MÉTRICAS
# ====================================================
def mostrar_metricas(y_real, y_pred, idioma=""):
    precision = precision_score(y_real, y_pred)
    recall = recall_score(y_real, y_pred)
    f1 = f1_score(y_real, y_pred)
    
    print(f"--- Métricas {idioma} ---")
    print(f"Precisión: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")
    print("------------------------")
    return precision, recall, f1

def graficar_error(er_historial, idioma=""):
    plt.figure(figsize=(8,5))
    plt.plot(range(len(er_historial)), er_historial, marker='o')
    plt.title(f'Error promedio por época - {idioma}')
    plt.xlabel('Época')
    plt.ylabel('Error promedio (MSE)')
    plt.grid(True)
    plt.show()

# ====================================================
# ENTRENAMIENTO Y EVALUACIÓN - ESPAÑOL
# ====================================================
mlp_es = MLP_TODO(
    num_entradas=X_entrenamiento_es_vect.shape[1],
    num_neuronas_ocultas=64,   # puedes cambiar según tabla de práctica
    num_salidas=1,
    epochs=10,
    batch_size=32,
    learning_rate=0.1,
    normalizacion="none",
    inicializacion="xavier"
)

print("=== Entrenando MLP Español ===")
mlp_es.train(X_entrenamiento_es_vect, y_train_es)

# Predicción y métricas
predicciones_es = mlp_es.predict(X_prueba_es_vect)
mostrar_metricas(y_test_es, predicciones_es, idioma="Español")

# Graficar error vs épocas
graficar_error(mlp_es.errores_history, idioma="Español")

# ====================================================
# ENTRENAMIENTO Y EVALUACIÓN - INGLÉS
# ====================================================
mlp_en = MLP_TODO(
    num_entradas=X_entrenamiento_en_vect.shape[1],
    num_neuronas_ocultas=64,
    num_salidas=1,
    epochs=10,
    batch_size=32,
    learning_rate=0.1,
    normalizacion="none",
    inicializacion="xavier"
)

print("=== Entrenando MLP Inglés ===")
mlp_en.train(X_entrenamiento_en_vect, y_train_en)

# Predicción y métricas
predicciones_en = mlp_en.predict(X_prueba_en_vect)
mostrar_metricas(y_test_en, predicciones_en, idioma="Inglés")

# Graficar error vs épocas
graficar_error(mlp_en.errores_history, idioma="Inglés")


In [ ]:
# ====================================================
# EXPERIMENTOS CON TODAS LAS CONFIGURACIONES
# ====================================================

# Parámetros a experimentar según la práctica
neuronas_ocultas = [64, 128, 256]  # puedes agregar 512, 1024 si quieres
inicializaciones = ["xavier", "normal"]
tipos_vector = ["tf", "tfidf"]  # frecuencia de términos o TF-IDF
ngramas_posibles = [(1,1), (1,2)]  # unigramas, unigramas+bigramas
preprocesamientos = ["none"]  # ya hicimos preprocesamiento, puedes definir variantes si quieres
learning_rates = [0.01, 0.1, 0.5]
batch_sizes = [16, 32, 64]
epochs_posibles = [100, 200, 300]

# Diccionario para almacenar resultados
resultados = []

# ====================================================
# FUNCION PARA EJECUTAR EXPERIMENTO
# ====================================================
def ejecutar_experimento(X_train_raw, y_train, X_test_raw, y_test, idioma=""):
    for n_ocultas in neuronas_ocultas:
        for inicializacion in inicializaciones:
            for tipo_vec in tipos_vector:
                for ngramas in ngramas_posibles:
                    for lr in learning_rates:
                        for batch in batch_sizes:
                            for epocas in epochs_posibles:

                                # Vectorizar
                                X_train_vect, X_test_vect, vectorizador = vectorizar_textos(
                                    X_train_raw, X_test_raw,
                                    metodo=tipo_vec,
                                    ngramas=ngramas
                                )

                                # Crear MLP
                                mlp = MLP_TODO(
                                    num_entradas=X_train_vect.shape[1],
                                    num_neuronas_ocultas=n_ocultas,
                                    num_salidas=1,
                                    epochs=epocas,
                                    batch_size=batch,
                                    learning_rate=lr,
                                    normalizacion="none",
                                    inicializacion=inicializacion
                                )

                                print(f"\nEntrenando {idioma} | Neuronas {n_ocultas}, Inicial {inicializacion}, Vector {tipo_vec}, Ngramas {ngramas}, LR {lr}, Batch {batch}, Epochs {epocas}")
                                
                                # Entrenamiento
                                mlp.train(X_train_vect, y_train)

                                # Predicción
                                y_pred = mlp.predict(X_test_vect)

                                # Métricas
                                precision, recall, f1 = mostrar_metricas(y_test, y_pred, idioma=idioma)

                                # Guardar resultados
                                resultados.append({
                                    "idioma": idioma,
                                    "neuronas_ocultas": n_ocultas,
                                    "inicializacion": inicializacion,
                                    "vector": tipo_vec,
                                    "ngramas": ngramas,
                                    "learning_rate": lr,
                                    "batch_size": batch,
                                    "epochs": epocas,
                                    "precision": precision,
                                    "recall": recall,
                                    "f1": f1,
                                    "error_final": mlp.errores_history[-1]
                                })

# ====================================================
# EJECUTAR PARA ESPAÑOL
# ====================================================
ejecutar_experimento(X_entrenamiento_es, y_train_es, X_prueba_es, y_test_es, idioma="Español")

# ====================================================
# EJECUTAR PARA INGLÉS
# ====================================================
ejecutar_experimento(X_entrenamiento_en, y_train_en, X_prueba_en, y_test_en, idioma="Inglés")

# ====================================================
# CONVERTIR RESULTADOS A DATAFRAME
# ====================================================
import pandas as pd
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by="f1", ascending=False)  # Ordenar por F1-score
print("\n--- TOP 5 CONFIGURACIONES ---")
print(df_resultados.head(5))


In [ ]:
# ====================================================
# GRAFICAR CURVAS DE ERROR DE LAS 5 MEJORES CONFIGURACIONES
# ====================================================

import matplotlib.pyplot as plt

# Seleccionar las 5 mejores configuraciones según F1-score
top5_config = df_resultados.head(5)

plt.figure(figsize=(10,6))

for i, fila in top5_config.iterrows():
    # Recuperar historial de errores del MLP de esa configuración
    # 🔹 Asumimos que guardaste los objetos MLP en resultados con clave 'mlp_obj'
    mlp_obj = fila['mlp_obj']
    plt.plot(range(len(mlp_obj.errores_history)), mlp_obj.errores_history, marker='o', label=(
        f"{fila['idioma']} | Noc:{fila['neuronas_ocultas']} | Init:{fila['inicializacion']} | "
        f"Vec:{fila['vector']} | LR:{fila['learning_rate']} | Batch:{fila['batch_size']} | Ep:{fila['epochs']}"
    ))

plt.title("Error promedio por época - Top 5 configuraciones")
plt.xlabel("Época")
plt.ylabel("Error promedio (MSE)")
plt.grid(True)
plt.legend(fontsize=8)
plt.show()
